In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import EfficientNetB3, ResNet50

In [2]:
base_dir = '../data/FER2013'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(
      rescale=1/255,    
#     rotation_range=40,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True
)

validation_datagen = ImageDataGenerator(
    rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=50,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=100,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
# pretrained network
model_base = EfficientNetB3(include_top=False, 
                            weights='imagenet', input_shape=(48,48,3))
# model_base의 weight학습을 동결
model_base.trainable=False

model_base.summary()

In [ ]:
model = Sequential()

# pretrained network를 우리의 모델 앞에 추가
model.add(model_base)

model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

model.summary()

model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, steps_per_epoch=48, epochs=30,
                    validation_data=validation_generator,
                    validation_steps=20)

In [4]:
## 다른 모델

model = Sequential()
model.add(Conv2D(64, (1, 1), padding='same', activation='relu', 
                 input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(256, (5, 5),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2),padding="same"))
model.add(Dropout(0.25))

model.add(Conv2D(256, (5, 5),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2),padding="same"))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))
model.summary()

model.compile(optimizer=Adam(learning_rate=1e-3), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 48, 48, 64)        128       
_________________________________________________________________
batch_normalization_5 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 128)       73856     
_________________________________________________________________
batch_normalization_6 (Batch (None, 48, 48, 128)       512       
_________________________________________________________________
dropout_6 (Dropout)          (None, 48, 48, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 48, 48, 256)      

In [5]:
history = model.fit(train_generator, steps_per_epoch=280, epochs=20,
                    validation_data=validation_generator,
                    validation_steps=71)

Epoch 1/20
280/280 [==============================] - 26s 92ms/step - loss: 1.8449 - accuracy: 0.2917 - val_loss: 1.8520 - val_accuracy: 0.2494
Epoch 2/20
280/280 [==============================] - 25s 89ms/step - loss: 1.5612 - accuracy: 0.3941 - val_loss: 1.4877 - val_accuracy: 0.4328
Epoch 3/20
280/280 [==============================] - 26s 91ms/step - loss: 1.3741 - accuracy: 0.4699 - val_loss: 1.2615 - val_accuracy: 0.5110
Epoch 4/20
280/280 [==============================] - 26s 92ms/step - loss: 1.2659 - accuracy: 0.5182 - val_loss: 1.2165 - val_accuracy: 0.5337
Epoch 5/20
280/280 [==============================] - 26s 92ms/step - loss: 1.1928 - accuracy: 0.5523 - val_loss: 1.1900 - val_accuracy: 0.5497
Epoch 6/20
280/280 [==============================] - 26s 91ms/step - loss: 1.1204 - accuracy: 0.5798 - val_loss: 1.2028 - val_accuracy: 0.5500
Epoch 7/20
280/280 [==============================] - 26s 92ms/step - loss: 1.0592 - accuracy: 0.6041 - val_loss: 1.1525 - val_accuracy: